In [24]:
TABLE1_PATH = "../data/public/main1.csv"
TABLE2_PATH = "../data/public/main2.csv"
TABLE3_PATH = "../data/public/main3.csv"

In [42]:
import pandas as pd

LIMIT_SIZE = 10000
table1 = pd.read_csv(TABLE1_PATH, nrows=LIMIT_SIZE)
table2 = pd.read_csv(TABLE2_PATH, nrows=LIMIT_SIZE)
table3 = pd.read_csv(TABLE3_PATH, nrows=LIMIT_SIZE)

table1.to_csv("../data/subset_main1.csv", index=False)
table2.to_csv("../data/subset_main2.csv", index=False)
table3.to_csv("../data/subset_main3.csv", index=False)

In [31]:
print(f"Table 1 columns: {list(table1.columns)}")
print(f"Table 2 columns: {list(table2.columns)}")
print(f"Table 3 columns: {list(table3.columns)}")

Table 1 columns: ['uid', 'full_name', 'email', 'address', 'sex', 'birthdate', 'phone']
Table 2 columns: ['uid', 'first_name', 'middle_name', 'last_name', 'birthdate', 'phone', 'address']
Table 3 columns: ['uid', 'name', 'email', 'birthdate', 'sex']


In [32]:
total_columns = ["uid", "name", "birthdate", "phone", "address", "sex", "email"]

In [36]:
import mapply

mapply.init(
    n_workers=-1,
    chunk_size=100,
    max_chunks_per_worker=8,
    progressbar=False,
)

## Preprocess Table1

In [37]:
table1.head(10)

,uid,full_name,email,address,sex,birthdate,phone
0,8357db00-b4ce-4c33-a66e-1842744d0ce4,БАЛКИБАЕВ Углы МАМАСАИД КАРЛИТО,mamasaid_balkibaev0@example.ru,"г. Урус-Мартан, пер. Шмидта, д. 233 к. 6/4, 63...",m,1954-06-15,8 (020) 645-0526
1,a40182ff-7b3c-4fe4-9455-505414bf6f1c,ИГИНОВА ЮТТА КОНДРАТЬЕВНП,jutta_iginova1@yandex.ru,"бул. Есенина, д. 2/3 стр. 180, 881977",f,1981-11-24,8 (197) 128-63-27
2,2b401a4a-33c0-47e0-b527-300e17b3a8da,МИРОДИЛОВ ЗАФЕР РОБЕРТ,zafer_mirodilov2@mail.ru,"ст. Красногорск (Моск.), ул. Урожайная, д. 4/9...",m,1954-07-03,+7 585 762 7855
3,a844b52d-6fc8-4aac-9180-092cf3cba7b0,5МИРОДИЛОВ ЗАФЕР РОБЕРТ,zafer_mirodilov2mail,"ст. Красногорск (Моск.), ул. Урожайная, д. 4/9...",m,954-01-01,+7 (585) 762 7855
4,95b72c05-1620-4f55-b3d4-c5151ff83184,БОРОВАЯ ШАХЗОДА НИКОЛАЕВНА,shahzoda_borovaja3@gmail.com,"к. Устюжна, ш. Инженерное, д. 1/7 к. 4, 782855",f,1989-07-15,+7 017 511 8662
5,6345b698-10e6-4f73-be20-ba6f6bed61a9,КУРБОНОВА НВРСУЛУ ФНИАС,nursulu_kurbonova4@example.org,"клх Кулу, пер. Кирова, д. 8 стр. 51, 713661",f,2000-10-21,+7 (180) 759-21-51
6,465ee3ba-1585-4bb6-90a7-f44deada619a,БЕТТИНШНР АННА АЛЕКСАНДРЛВНА,anna_bettinger5@yandex.ru,"г. Шахты, ул. Павлика Морозова, д. 3/7, 304052",f,2002,83121835172
7,ad64731a-1034-4e03-a131-6005ecc98530,КРИЛОВ Оглы ЮРИЙ ЯКУП,jurij_krilov6@example.org,"с. Солнечногорск, ш. Краснопартизанское, д. 83...",m,1965-01-01,+7254116-65-76
8,bec6df67-5031-442a-9ec8-35a4e0d9ce61,KRI-LOV \nРИЙ ЯKУП,jurih_кrilov6zexamplеlоrg,"с. Солнечногорсу, ш. Краснлпарьижмнскоп, д. э3...",m,1965,6-65-76
9,2cb36637-9214-4c3a-bc0c-c2a8f232f121,ЙОХАНСЕН углы АЙЖАМАЛ ИВАНОВНА,ajzhamal_johansen7@mail.ru,"к. Тарко-Сале, бул. Ленинградский, д. 1 стр. 3...",f,2982-10-26,+7 964 990-96-24


In [ ]:
def preprocess_table1(df: pd.DataFrame) -> pd.DataFrame:
    df["name"] = df["full_name"].mapply(lambda x: x.replace("\n", " "))
    return df[total_columns]

# Preprocess Table2

In [38]:
table2.head(10)

,uid,first_name,middle_name,last_name,birthdate,phone,address
0,fd0eb198-eeef-4ff9-a983-26dd9aebb084,МАМАСАИД,КАРЛИТО,БAЛKИБАEВ,1954-06-15,8 (020) 6450526,"г. Урус-Мартан, пер. Шмидта, д. 233 к. 6/4, 63..."
1,7ac130fe-690e-4e39-9b25-b141095f3eec,ЗАФЕР,РОБЕРТ,МИИРОДИЛОВ,1954-07-03,+7 (585) 762 7855,"ст. Красногорск (Моск.), ул. Урожайная, д. 4/9..."
2,f564db39-b7b1-4643-8d24-4759d5ba6ba5,ШАХЗОДА,НИКОЛАЕВНА,БОРОВАЯ,1989-07-15,511-86-62,"к. Устюжна, ш. Инженерное, д. 1/7 к. 4, 782855"
3,93c32877-6cf0-48f9-bb1e-c67794f08541,НУРСУЛУ,Неетт,КУРБОНОВА,2000-10-21,+7 (180) 759-21-51,"клх Кулу, пер. Кирова, д. 8 стр. 51, 713661"
4,a207993d-185f-4b86-bda6-6ce2c0c6a0ff,ЮРРИЙ,ЯКУП,КРИЛОВ,1965-05-31,66576,"с. Солнечногорск, ш. Краснопартизанское, д. 83..."
5,33b21a5e-de52-4e07-a209-0dbaab1f5dd3,АЙЖАМАЛ,ИВАНОВНА,ЙОХАНСЕН,1982-10-26,+7 964 990 96 24,"к. Тарко-Сале, бул. Ленинградский, д. 1 стр. 3..."
6,accb983e-1d0c-47ea-b633-fc9ba05fdc00,ГЕНО,ВЛАДИМИРОВИЧ,САЛОКОСКИ,1960-07-06,8 302 009 64 37,"г. Кудымкар, бул. Л.Толстого, д. 592, 617939"
7,adfc5dbd-d7af-4df5-bb5f-b7ba553e9668,КЮЛЛИККИ,АЛЕКСАНДРОВНА,УОИДЛИ,1978-01-10,+7 (180) 710-09-77,"к. Яшалта, пер. Азина, д. 6/3 стр. 3/7, 726128"
8,4fbedf43-c37d-4498-b20e-a2f36429247b,ВЛАДИСЛАВ,ФЕЛИКСОВИЧ,LENDEKI,975-06-02,+7 347 232-9206,"г. Тула, пер. Ярославский, д. 368 к. 2/2, 500850"
9,9a23a9ed-f2a4-4054-9dc9-501d2b98f50d,ЖАХОНГИР,ЧAРЛЬЗ,Нет,1989-08-14,3 5976,"д. Токсово, наб. Маяковского, д. 2 стр. 2/6, 2..."


In [39]:
def preprocess_table2(df: pd.DataFrame) -> pd.DataFrame:
    df["name"] = df["last_name"] + " " + df["first_name"] + " " + df["middle_name"]
    return df[total_columns]

## Preprocess Table3

In [41]:
table3.head(20)

,uid,name,email,birthdate,sex
0,9185f879-6289-4ad6-9cb0-ed7cb6cb652b,МАМАСАИД ЬАДКИБАЕВ,mamasaid_balkibaev0@example.ru,9145-06-15,m
1,289fcf9e-af05-43cf-a6d4-1d1d3cf5131f,ЮТТА ИГИНОВА,jutta_ihinova1;yanddx,1981-11-24,f
2,c2b04755-ebd8-4625-b8bb-4131ca049415,ЗАФЕР МИРОДИЛОВ,zafer_mirodilov2@mail.ru,1954,m
3,49978ae8-51fe-4a74-908e-22ec9e693855,ШАХЗОДА БОРОВАЯ,shahzoda_borovaja3@gmail.com,2989-07-15,f
4,75e199c5-86fa-4d3b-b1ea-783ecc26b021,НУРСУЛУ КУРБОНОВА,nursulu_kurbonova4@example.org,2000-10-21,f
5,8743ab3c-260b-4911-8ff9-f10187e1ab7c,АННА БЕТТИНГЕР Огли,anna_bettinger5@yandex.ru,1002-04-02,f
6,2e49b72b-487d-4f17-835d-0a04051bc654,ЮРИЙ КРИЛОВ,jurij_krilov6@example.org,1965-05-31,m
7,265e4734-ba29-432c-852c-e29c941568e6,АЙЖАМАЛ ЙОХАНСЕН,ajzhamal_johansen7@mail.ru,982-10-26,f
8,d491b555-9507-43e0-a3e3-8a0071ec7761,ГЕНО САЛОКОСКИ,geno_salokoski8@example.com,1960-07-06,m
9,6d875c40-e409-479b-807e-13ee3d7ca445,КЮЛЛИККИ УОИДЛИ,kjullikki_uoidli9@gmail,1978-01-10,f


In [ ]:
def preprocess_table3(df: pd.DataFrame) -> pd.DataFrame:
    df["name"] = df["last_name"] + " " + df["first_name"] + " " + df["middle_name"]
    return df[total_columns]

In [ ]:
import recordlinkage

indexer = recordlinkage.Index()
indexer.block("surname")
candidate_links = indexer.index(table1, table2)

In [ ]:
c = recordlinkage.Compare()

c.string("name_a", "name_b", method="jarowinkler", threshold=0.85)
c.exact("sex", "gender")
c.date("dob", "date_of_birth")
c.string("str_name", "streetname", method="damerau_levenshtein", threshold=0.7)
c.exact("place", "placename")
c.numeric("income", "income", method="gauss", offset=3, scale=3, missing_value=0.5)

# The comparison vectors
feature_vectors = c.compute(candidate_links, df_a, df_b)